In [1]:
import pennylane as qml
import torch
import numpy as np
import random
import torch.nn as nn
from tqdm import tqdm

In [2]:
import functools
import inspect
import math
from collections.abc import Iterable
from typing import Callable, Dict, Union, Any

from pennylane.qnode import QNode

try:
    import torch
    from torch.nn import Module

    TORCH_IMPORTED = True
except ImportError:
    # The following allows this module to be imported even if PyTorch is not installed. Users
    # will instead see an ImportError when instantiating the TorchLayer.
    from unittest.mock import Mock

    Module = Mock
    TORCH_IMPORTED = False


class TorchLayer(Module):
    def __init__(self,qnode,weights):
        if not TORCH_IMPORTED:
            raise ImportError(
                "TorchLayer requires PyTorch. PyTorch can be installed using:\n"
                "pip install torch\nAlternatively, "
                "visit https://pytorch.org/get-started/locally/ for detailed "
                "instructions."
            )
        super().__init__()

        #weight_shapes = {
        #    weight: (tuple(size) if isinstance(size, Iterable) else () if size == 1 else (size,))
        #    for weight, size in weight_shapes.items()
        #}

        # validate the QNode signature, and convert to a Torch QNode.
        # TODO: update the docstring regarding changes to restrictions when tape mode is default.
        #self._signature_validation(qnode, weight_shapes)
        self.qnode = qnode
        self.qnode.interface = "torch"

        self.qnode_weights = weights

    def forward(self, inputs):  # pylint: disable=arguments-differ
        """Evaluates a forward pass through the QNode based upon input data and the initialized
        weights.

        Args:
            inputs (tensor): data to be processed

        Returns:
            tensor: output data
        """

        if len(inputs.shape) > 1:
            # If the input size is not 1-dimensional, unstack the input along its first dimension,
            # recursively call the forward pass on each of the yielded tensors, and then stack the
            # outputs back into the correct shape
            reconstructor = [self.forward(x) for x in torch.unbind(inputs)]
            return torch.stack(reconstructor)

        # If the input is 1-dimensional, calculate the forward pass as usual
        return self._evaluate_qnode(inputs)


    def _evaluate_qnode(self, x):
        """Evaluates the QNode for a single input datapoint.

        Args:
            x (tensor): the datapoint

        Returns:
            tensor: output datapoint
        """
        kwargs = {
            **{self.input_arg: x},
            **{arg: weight.to(x) for arg, weight in self.qnode_weights.items()},
        }
        res = self.qnode(**kwargs)

        if isinstance(res, torch.Tensor):
            return res.type(x.dtype)

        return torch.hstack(res).type(x.dtype)

    def __str__(self):
        detail = "<Quantum Torch Layer: func={}>"
        return detail.format(self.qnode.func.__name__)

    __repr__ = __str__
    _input_arg = "inputs"

    @property
    def input_arg(self):
        """Name of the argument to be used as the input to the Torch layer. Set to ``"inputs"``."""
        return self._input_arg

In [188]:
class QSAL_pennylane(torch.nn.Module):
    def __init__(self,S,n,Denc,D):
        super().__init__()
        self.seq_num=S
        self.init_params_Q=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_K=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_V=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.num_q=n
        self.Denc=Denc
        self.D=D
        self.d=n*(Denc+2)
        self.dev = qml.device("default.qubit", wires=self.num_q)
        
        self.vqnod=qml.QNode(self.circuit_v, self.dev, interface="torch")
        self.qnod=qml.QNode(self.circuit_qk, self.dev, interface="torch")
        self.weight_v = [{"weights": self.init_params_V[i]} for i in range(self.seq_num)]
        self.weight_q = [{"weights": self.init_params_Q[i]} for i in range(self.seq_num)]
        self.weight_k = [{"weights": self.init_params_K[i]} for i in range(self.seq_num)]
        #self.v_linear ={} #[qml.qnn.TorchLayer(self.vqnod[i], self.weight_shapes) for i in range(self.seq_num)]
        #for i in range(self.seq_num):
        self.v_linear = [TorchLayer(self.vqnod, self.weight_v[i]) for i in range(self.seq_num)]
        self.q_linear = [TorchLayer(self.qnod, self.weight_q[i]) for i in range(self.seq_num)]
        self.k_linear = [TorchLayer(self.qnod, self.weight_k[i]) for i in range(self.seq_num)]
        #self.qqnod=[qml.QNode(self.circuit_qk, self.dev, interface="torch") for i in range(self.seq_num)]

    def random_op(self):
        a=random.randint(0, 4)
        if a==0:
            op=qml.Identity(0)
        elif a==1:
            op=qml.PauliX(0)
        elif a==2:
            op=qml.PauliY(0)
        else:
            op=qml.PauliZ(0)

        op_elimated=qml.Identity(0)
        for i in range(1,self.num_q):
            op_elimated=op_elimated@qml.Identity(i)
        Select_wrong=True
        while Select_wrong:
            for i in range(1,self.num_q):
                a=random.randint(0, 4)
                if a==0:
                    op=op@qml.Identity(i)
                elif a==1:
                    op=op@qml.PauliX(i)
                elif a==2:
                    op=op@qml.PauliY(i)
                else:
                    op=op@qml.PauliZ(i)
            if op!=op_elimated:
                Select_wrong=False
        return op

    def circuit_v(self,inputs,weights):
            op=self.random_op()
            # feature_map
            indx=0
            for j in range(self.num_q):
                qml.RX(inputs[indx],j)
                qml.RY(inputs[indx+1],j)
                indx+=2
            for i in range(self.Denc):
                for j in range(self.num_q):
                    qml.CNOT(wires=(j,(j+1)%self.num_q))

                for j in range(self.num_q):
                    qml.RY(inputs[indx],j)
                    indx+=1
            # Ansatz
            indx=0
            for j in range(self.num_q):
                qml.RX(weights[indx],j)
                qml.RY(weights[indx+1],j)
                indx+=2
            for i in range(self.D):
                for j in range(self.num_q):
                    qml.CNOT(wires=(j,(j+1)%self.num_q))
                    
                for j in range(self.num_q):
                    #qc.rx(params[indx],j)
                    qml.RY(weights[indx],j)
                    indx+=1
            return [qml.expval(op) for i in range(self.d)] 

    def circuit_qk(self,inputs,weights):
        op=self.random_op()
        # feature_map
        indx=0
        for j in range(self.num_q):
            qml.RX(inputs[indx],j)
            qml.RY(inputs[indx+1],j)
            indx+=2
        for i in range(self.Denc):
            for j in range(self.num_q):
                qml.CNOT(wires=(j,(j+1)%self.num_q))

            for j in range(self.num_q):
                qml.RY(inputs[indx],j)
                indx+=1
        # Ansatz
        indx=0
        for j in range(self.num_q):
            qml.RX(weights[indx],j)
            qml.RY(weights[indx+1],j)
            indx+=2
        for i in range(self.D):
            for j in range(self.num_q):
                qml.CNOT(wires=(j,(j+1)%self.num_q))
                
            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qml.RY(weights[indx],j)
                indx+=1
        return [qml.expval(qml.PauliZ(0))]

    def forward(self,input):

        Q_output=torch.stack([self.q_linear[i](input[:,i]) for i in range(self.seq_num)])
        K_output=torch.stack([self.k_linear[i](input[:,i]) for i in range(self.seq_num)])
        V_output=torch.stack([self.v_linear[i](input[:,i]) for i in range(self.seq_num)])
        
        batch_size=len(input)
        Q_output=Q_output.transpose(0,2).repeat((self.seq_num,1,1))
        K_output=K_output.transpose(0,2).repeat((self.seq_num,1,1)).transpose(0,2)
        #print(V_output.size())
        #Q_grid, K_grid=torch.meshgrid(Q_output, K_output, indexing='ij')
        alpha=torch.exp(-(Q_output-K_output)**2)
        alpha=alpha.transpose(0,1)
        V_output=V_output.transpose(0,1)
        output=[]

        for i in range(self.seq_num):
            
            Sum_a=torch.sum(alpha[:,i,:],-1)
            div_sum_a=(1/Sum_a).repeat(self.d,self.seq_num,1).transpose(0,2)
            
            Sum_w=torch.sum(alpha[:,:,i].repeat((self.d,1,1)).transpose(0,2).transpose(0,1)*V_output*div_sum_a,1)
            output.append(Sum_w)
        return input+torch.stack(output).transpose(0,1)

class QSANN_pennylane(torch.nn.Module):
    def __init__(self,S,n,Denc,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding 
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.qsal_lst=[QSAL_pennylane(S,n,Denc,D) for _ in range(num_layers)]
        self.qnn=nn.Sequential(*self.qsal_lst)

    def forward(self,input):

        return self.qnn(input)

class QSANN_text_classifier(torch.nn.Module):
    def __init__(self,S,n,Denc,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding 
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.Qnn=QSANN_pennylane(S,n,Denc,D,num_layers)
        self.final_layer=nn.Linear(n*(Denc+2)*S, 10)
        self.final_layer=self.final_layer.float()

    def forward(self,input):

        x=self.Qnn(input)
        x=torch.flatten(x, start_dim=1)
        
        return self.final_layer(x)

# Multiclass Classification with Sklearn Image Dataset

# Sklearn Image Dataset (Patches prepared Row-wise)

In [240]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
digits = load_digits()
X, y = digits.images[0:200], digits.target[0:200]
X_train, X_test, y_train, y_test = train_test_split(X, y)

#train_mask = np.isin(y_train, [1, 7])
#X_train, y_train = X_train[train_mask], y_train[train_mask]

#test_mask = np.isin(y_test, [1, 7])
#X_test, y_test = X_test[test_mask], y_test[test_mask]

X_train = X_train.reshape(X_train.shape[0], 4, 16)
X_test = X_test.reshape(X_test.shape[0], 4, 16)

In [241]:
from keras.utils import to_categorical

In [242]:
y_train

array([5, 3, 7, 1, 2, 2, 3, 5, 5, 7, 6, 3, 2, 0, 8, 4, 8, 5, 3, 4, 0, 7,
       0, 9, 6, 6, 3, 4, 1, 9, 0, 6, 5, 5, 8, 7, 8, 7, 7, 5, 1, 9, 8, 8,
       8, 3, 2, 5, 0, 0, 0, 2, 6, 6, 3, 1, 1, 2, 6, 8, 3, 8, 8, 4, 8, 4,
       3, 0, 9, 9, 3, 9, 9, 7, 0, 4, 3, 9, 9, 0, 6, 1, 5, 4, 0, 5, 3, 2,
       8, 6, 8, 1, 9, 0, 6, 1, 3, 2, 4, 6, 6, 2, 3, 6, 6, 3, 8, 1, 9, 6,
       6, 5, 2, 1, 6, 9, 7, 2, 1, 9, 5, 0, 2, 9, 0, 1, 1, 3, 4, 7, 0, 0,
       0, 4, 9, 8, 4, 1, 2, 4, 3, 8, 5, 2, 0, 9, 8, 0, 6, 7])

In [243]:
y_train = to_categorical(y_train)

In [245]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [234]:
y_test = to_categorical(y_test)

In [227]:
model=QSANN_text_classifier(4, 4, 2, 1, 1)

# seq * (num_qubits) * (num_layers + 2)

In [228]:
model

QSANN_text_classifier(
  (Qnn): QSANN_pennylane(
    (qnn): Sequential(
      (0): QSAL_pennylane()
    )
  )
  (final_layer): Linear(in_features=64, out_features=10, bias=True)
)

In [229]:
optimizer = torch.optim.Adam(lr=0.001, params=model.parameters())

In [230]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

794


In [231]:
criterion = torch.nn.CrossEntropyLoss()

In [232]:
def multi_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    pred_soft = torch.log_softmax(preds, dim=1)
    _, pred_index = torch.max(pred_soft, dim=1)
    print(pred_index)
    y = [int(w.argmax()) for w in y]
    y = torch.tensor(y, dtype=float)
    print(y)
    correct_pred = (pred_index == y).float()
    #round predictions to the closest integer
    
    acc = correct_pred.sum()/len(correct_pred)

    return acc

In [233]:
for iepoch in tqdm(range(100)):
    optimizer.zero_grad()
    X_tensor=torch.tensor(X_train)
    predictions=model(X_tensor.float()).squeeze(1)
    #predictions=torch.sign(predictions)
    #print(predictions)
    label=torch.tensor(y_train)
    #for i in range(len(label)):
    #    if label[i]==1:
    #        label[i] = 0
    #    else:
    #        label[i]=1
    #print(label)
    loss = criterion(predictions, label.float())
    acc = multi_accuracy(predictions, label)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()

  0%|          | 0/100 [00:00<?, ?it/s]

tensor([8, 7, 3, 4, 3, 8, 3, 3, 3, 3, 3, 8, 3, 3, 8, 3, 7, 3, 8, 3, 8, 3, 8, 3,
        8, 4, 3, 3, 8, 3, 7, 8, 7, 3, 7, 3, 7, 3, 3, 8, 3, 3, 3, 3, 8, 7, 8, 3,
        3, 4, 8, 5, 5, 7, 3, 3, 5, 3, 8, 3, 8, 8, 3, 7, 3, 3, 3, 3, 8, 3, 8, 8,
        5, 5, 8])
tensor([0., 1., 0., 2., 3., 0., 3., 3., 0., 1., 1., 0., 3., 1., 7., 5., 4., 4.,
        7., 0., 6., 9., 0., 5., 6., 2., 3., 8., 2., 2., 7., 4., 1., 8., 7., 3.,
        5., 0., 7., 0., 9., 9., 5., 3., 6., 7., 8., 5., 1., 4., 6., 9., 9., 7.,
        2., 1., 9., 0., 6., 0., 4., 6., 3., 1., 8., 5., 3., 2., 6., 2., 7., 8.,
        9., 9., 6.], dtype=torch.float64)

Accuracy: tensor(0.2133)

tensor(7.0260, grad_fn=<DivBackward1>)


  1%|          | 1/100 [00:29<49:18, 29.89s/it]

tensor([8, 7, 3, 4, 3, 8, 3, 3, 3, 3, 3, 8, 3, 3, 8, 3, 7, 7, 8, 3, 8, 3, 8, 3,
        8, 4, 3, 3, 4, 3, 7, 8, 7, 3, 7, 3, 7, 3, 7, 8, 3, 3, 3, 3, 8, 7, 8, 7,
        7, 4, 8, 5, 5, 7, 3, 3, 5, 3, 8, 3, 8, 8, 3, 7, 3, 3, 3, 3, 8, 4, 8, 8,
        5, 5, 8])
tensor([0., 1., 0., 2., 3., 0., 3., 3., 0., 1., 1., 0., 3., 1., 7., 5., 4., 4.,
        7., 0., 6., 9., 0., 5., 6., 2., 3., 8., 2., 2., 7., 4., 1., 8., 7., 3.,
        5., 0., 7., 0., 9., 9., 5., 3., 6., 7., 8., 5., 1., 4., 6., 9., 9., 7.,
        2., 1., 9., 0., 6., 0., 4., 6., 3., 1., 8., 5., 3., 2., 6., 2., 7., 8.,
        9., 9., 6.], dtype=torch.float64)

Accuracy: tensor(0.2267)

tensor(6.6618, grad_fn=<DivBackward1>)


  2%|▏         | 2/100 [01:01<50:25, 30.88s/it]

...

 98%|█████████▊| 98/100 [54:42<01:07, 33.55s/it]

tensor([0, 1, 0, 2, 3, 0, 3, 3, 0, 1, 1, 0, 3, 1, 7, 5, 4, 4, 7, 0, 6, 9, 0, 5,
        6, 2, 3, 8, 2, 3, 7, 4, 1, 8, 7, 3, 9, 0, 7, 0, 9, 9, 5, 3, 6, 7, 8, 5,
        1, 4, 6, 9, 9, 7, 0, 1, 9, 0, 6, 0, 4, 6, 3, 1, 8, 5, 3, 2, 6, 2, 7, 8,
        9, 9, 6])
tensor([0., 1., 0., 2., 3., 0., 3., 3., 0., 1., 1., 0., 3., 1., 7., 5., 4., 4.,
        7., 0., 6., 9., 0., 5., 6., 2., 3., 8., 2., 2., 7., 4., 1., 8., 7., 3.,
        5., 0., 7., 0., 9., 9., 5., 3., 6., 7., 8., 5., 1., 4., 6., 9., 9., 7.,
        2., 1., 9., 0., 6., 0., 4., 6., 3., 1., 8., 5., 3., 2., 6., 2., 7., 8.,
        9., 9., 6.], dtype=torch.float64)

Accuracy: tensor(0.9600)

tensor(0.2170, grad_fn=<DivBackward1>)


 99%|█████████▉| 99/100 [55:16<00:33, 33.54s/it]

tensor([0, 1, 0, 2, 3, 0, 3, 3, 0, 1, 1, 0, 3, 1, 7, 5, 4, 4, 7, 0, 6, 9, 0, 5,
        6, 2, 3, 8, 2, 3, 7, 4, 1, 8, 7, 3, 9, 0, 7, 0, 9, 9, 5, 3, 6, 7, 8, 5,
        1, 4, 6, 9, 9, 7, 0, 1, 9, 0, 6, 0, 4, 6, 3, 1, 8, 5, 3, 2, 6, 2, 7, 8,
        9, 9, 6])
tensor([0., 1., 0., 2., 3., 0., 3., 3., 0., 1., 1., 0., 3., 1., 7., 5., 4., 4.,
        7., 0., 6., 9., 0., 5., 6., 2., 3., 8., 2., 2., 7., 4., 1., 8., 7., 3.,
        5., 0., 7., 0., 9., 9., 5., 3., 6., 7., 8., 5., 1., 4., 6., 9., 9., 7.,
        2., 1., 9., 0., 6., 0., 4., 6., 3., 1., 8., 5., 3., 2., 6., 2., 7., 8.,
        9., 9., 6.], dtype=torch.float64)

Accuracy: tensor(0.9600)

tensor(0.2096, grad_fn=<DivBackward1>)


100%|██████████| 100/100 [55:49<00:00, 33.50s/it]


In [236]:
X_tensor=torch.tensor(X_test)
predictions=model(X_tensor.float()).squeeze(1)
label=torch.tensor(y_test)

loss = criterion(predictions, label.float())
acc = multi_accuracy(predictions, label.float())
print('')
print('Accuracy:',acc)
print('')
print(loss)

tensor([2, 6, 5, 5, 4, 5, 3, 1, 8, 7, 3, 9, 1, 6, 5, 2, 5, 7, 4, 8, 6, 3, 6, 1,
        2])
tensor([8., 1., 4., 5., 4., 2., 3., 1., 8., 7., 3., 9., 1., 6., 5., 2., 5., 7.,
        4., 8., 6., 3., 6., 1., 2.], dtype=torch.float64)

Accuracy: tensor(0.8400)

tensor(0.6853, grad_fn=<DivBackward1>)


# Sklearn Image Dataset (Patches Prepared Column-wise)

In [83]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
digits = load_digits()
X, y = digits.images[0:500], digits.target[0:500]
X_train, X_test, y_train, y_test = train_test_split(X, y)

train_mask = np.isin(y_train, [1, 7])
X_train, y_train = X_train[train_mask], y_train[train_mask]

test_mask = np.isin(y_test, [1, 7])
X_test, y_test = X_test[test_mask], y_test[test_mask]

#X_train = X_train.reshape(X_train.shape[0], 4, 16)
#X_test = X_test.reshape(X_test.shape[0], 4, 16)
X_train_c = X_train.transpose(0, 2, 1)
X_test_c = X_test.transpose(0, 2, 1)

X_train_c = X_train_c.reshape(X_train.shape[0], 4, 16)
X_test_c = X_test_c.reshape(X_test.shape[0], 4, 16)

In [79]:
model=QSANN_text_classifier(4,4,2,1,1)
optimizer = torch.optim.Adam(lr=0.01, params=model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [81]:
for iepoch in tqdm(range(10)):
    optimizer.zero_grad()
    X_tensor=torch.tensor(X_train_c)
    predictions=model(X_tensor.float()).squeeze(1)
    #predictions=torch.sign(predictions)
    #print(predictions)
    label=torch.tensor(y_train)
    for i in range(len(label)):
        if label[i]==1:
            label[i] = 0
        else:
            label[i]=1
    #print(label)
    loss = criterion(predictions, label.float())
    acc = binary_accuracy(predictions, label)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()


  0%|          | 0/10 [00:00<?, ?it/s]


Accuracy: tensor(0.9481)

tensor(146.3840, grad_fn=<DivBackward1>)



 10%|█         | 1/10 [00:24<03:43, 24.87s/it]


Accuracy: tensor(0.9481)

tensor(145.6917, grad_fn=<DivBackward1>)



 20%|██        | 2/10 [00:50<03:21, 25.24s/it]

...


 80%|████████  | 8/10 [03:28<00:53, 26.82s/it]


Accuracy: tensor(0.9870)

tensor(142.2015, grad_fn=<DivBackward1>)



 90%|█████████ | 9/10 [03:53<00:26, 26.36s/it]


Accuracy: tensor(1.)

tensor(142.0063, grad_fn=<DivBackward1>)


100%|██████████| 10/10 [04:19<00:00, 25.91s/it]


In [84]:
X_tensor=torch.tensor(X_test_c)
predictions=model(X_tensor.float()).squeeze(1)
print(predictions)
label=torch.tensor(y_test)
for i in range(len(label)):
        if label[i]==1:
            label[i] = 0
        else:
            label[i]=1
print(label)
loss = criterion(predictions, label.float())
acc = binary_accuracy(predictions, label.float())
print('')
print('Accuracy:',acc)
print('')
print(loss)

tensor([5.2172e-03, 9.9299e-01, 9.9714e-01, 2.4885e-02, 3.4008e-03, 1.7592e-03,
        9.9949e-01, 3.8367e-03, 9.9953e-01, 9.9986e-01, 3.1252e-03, 8.2579e-02,
        9.9762e-01, 9.9997e-01, 3.1298e-03, 7.1565e-03, 9.9605e-01, 9.9876e-01,
        9.9963e-01, 1.4773e-03, 9.2540e-04], grad_fn=<SqueezeBackward1>)
tensor([0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0])

Accuracy: tensor(1.)

tensor(26.4663, grad_fn=<DivBackward1>)
